# Chocolate Database analysis

The Open Food Facts is a collaborative project where individual can contribute by adding data from the food products they buy. This project is a practice for visualization and classification specificaly on chocolate products. Here is the list of ideas, questions I wanted to investigate:

As any chocolate lover knows, there are several types of chocolate bar based on the amount of actual cocoa in the product. My hypothesis is that those categories should have a direct influence on the nutrition values (fat, carbohydrates, protein, etc.). Hence the first idea is to create a model to classify the different types of chocolate bars.
Based on the identified types, see the distribution for the nutrients and the prevalence of certain categories by brands.
Map the countries of origin with a visual for the number of product of that origin.
Map the dominant type of chocolate for each consumer coutry.
With no further ado, let's get started!

## Data Loading and cleaning

In [1]:
from openfoodfacts import API, APIVersion, Country, Environment, Flavor
import time
import json
import csv
import pandas as pd #version 2.3.3
import numpy as np #version 2.3.1

In [2]:
api = API(
    user_agent="<application name>",
    username=None,
    password=None,
    country=Country.world,
    flavor=Flavor.off,
    version=APIVersion.v2,
    environment=Environment.org,
    timeout=10
)

In [3]:
size = 100 #Maximum allowed by the api
results = api.product.text_search(query="chocolate, cocoa", page=1, page_size=size)
count = results.get("count")
total_pages = int(np.round(count/size))

ReadTimeout: HTTPSConnectionPool(host='world.openfoodfacts.org', port=443): Read timed out. (read timeout=10.0)

Loading this first page allows to have a first sight on the data. It also gives the total products found and the number of pages to go through.
While iterating through each page of the search, only relevant data will be captured into the final dataframe.
Thus a first selection of the relevant columns is required.

In [ ]:
def filter_df_by_keywords(df, keywords):
    """
    Filters a DataFrame to return a list which includes only columns 
    whose names contain any of the given keywords (provided as a list).
    """
    relevant_cols = []
    for col in df.columns: 
        for keyword in keywords:
            if keyword.lower() in col.lower():
                relevant_cols.append(col)
    return relevant_cols

targeted_keywords = ["name", "Quantity", "Brands", "Categories", 
                    "Manufacturing", "Stores", "Countr", 
                     "Ingredients","Origin", "nutriments", "id",
                    "keywords"]

target_col = filter_df_by_keywords(df_test, targeted_keywords)

Columns number was narrowed down from 550 to 390 with this first keyword filtering.
Unfortunately, the remaining columns have very closely related names that can hardly be automaticaly filtered.
Hence, manual picking of columns was performed, aiming for information revelant for our analysis and provided in English.

In [ ]:
column_selection = ['_id', '_keywords', 'generic_name_en','product_name', 'categories',
    'categories_hierarchy', 'brands','quantity', 'ingredients_original_tags', 'ingredients_text_en',
    'nutriments', 'countries', 'stores', 'manufacturing_places', 'origin', 'origin_en']
len(column_selection)

This final selection has 16 elements though the nutriments are nested within dictionaries for each product.
Normalizing gives 349 extra columns. Columns with more than 20 % of Nan are to be dropped.
The columns will then be filtered to retain only information standardized by 100 g.

In [ ]:
def unpacking_nutriments(df, thershold):
    """
    Unpacking the nutriments, filterint relevant columns (100g an units),
    then merging it back with the initial dataframe

    Arguments: 
    - df: the dataframe containg a column nutriments where nutriments are stored in dictionaries
    - threshold: percentage of Nan within a column that triggers the removal of the column
    """
    df_nutriments = pd.json_normalize(df["nutriments"])
    
    df_nutriments = df_nutriments.dropna(axis=1, thresh= len(df_nutriments)*(thershold/100)) 
    
    targeted_keywords_2 = ["100g", "unit"]
    target_col_nutriments = filter_df_by_keywords(df_nutriments, targeted_keywords_2)
    # this specific column is not relevant for our analysis because there shouldn't be any vegetable in any respectable chocolate.
    target_col_nutriments.remove("fruits-vegetables-legumes-estimate-from-ingredients_100g") 

    df = pd.concat([df, df_nutriments[target_col_nutriments]], axis=1)
    df = df.drop(columns=["nutriments"])
    return df

df_test = unpacking_nutriments(df_test[column_selection], 20)
print(df_test.shape)

The final dataframe should have 43 columns.

In [ ]:
max_retries = 4
products = []

for i in range(56, total_pages + 1): # replace 5 by total_pages + 1 for definitive version
    for attempt in range(max_retries):
        try:
            results = api.product.text_search(query="chocolate, cocoa", page=i, page_size=size)
            product_list = results.get("products", [])

            # Filtering the column selected previously to more efficiently store data
            filtered_products = [
                {k: product.get(k) for k in column_selection if k in product}
                for product in product_list]
            
            products.extend(filtered_products)
            break
            
        except Exception as e:
            print(f"Error fetching page {i} (Attempt {attempt+1}/{max_retries}): {e}")
            time.sleep(10 + attempt * 2) # Exponential backoff: sleep longer on subsequent fails
            
    else: # This 'else' belongs to the inner 'for' loop and executes if 'break' wasn't hit
        
        print(f"Failed to fetch page {i} after {max_retries} attempts. Skipping.")
    #time.sleep(10)


In [ ]:
df = pd.DataFrame(products)
df = unpacking_nutriments(df, 20)
df.shape

In [ ]:
count

In [ ]:
#filepath = r'D:\OpenFoodFacts_chocolate.csv'
#df.to_csv(filepath)